<a href="https://colab.research.google.com/github/deondrae4088/WebMd_chat/blob/Project-3-(Gabe's-Branch)/Project3GabesBranch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade tensorflow kagglehub
import tensorflow as tf
import pathlib
import kagglehub
import os
import zipfile
import pathlib
import glob
import PIL

dataset_names = [
    "shubhamgoel27/dermnet",
    "pkdarabi/brain-tumor-image-dataset-semantic-segmentation",
    "paultimothymooney/chest-xray-pneumonia",
    "maedemaftouni/covid19-ct-scan-lesion-segmentation-dataset",
    "aryashah2k/breast-ultrasound-images-dataset"
]

extracted_dirs = []

for name in dataset_names:
    try:
        download_path = kagglehub.dataset_download(name)
        print(f"Downloaded '{name}' to: {download_path}")
        extract_path = os.path.join(os.getcwd(), name.split('/')[1]) if download_path.endswith(".zip") else download_path
        if download_path.endswith(".zip"):
            with zipfile.ZipFile(download_path, 'r') as zip_ref:
                zip_ref.extractall(extract_path)
            print(f"Extracted '{name}' to: {extract_path}")
        extracted_dirs.append(pathlib.Path(extract_path))
    except Exception as e:
        print(f"Error with '{name}': {e}")

[print(f"Processing: {img_dir}") for img_dir in extracted_dirs]

In [ ]:

all_images = []

def find_images(img_dir):
    print(f"Processing: {img_dir}")
    image_patterns = [str(img_dir / '**/*.jpg'), str(img_dir / '**/*.jpeg'), str(img_dir / '**/*.png')]
    dataset_images = [pathlib.Path(f) for pattern in image_patterns for f in glob.glob(pattern, recursive=True)]
    print(f"  Found {len(dataset_images)} images")
    return dataset_images

list_of_image_lists = list(map(find_images, extracted_dirs))
all_images = [img for sublist in list_of_image_lists for img in sublist]

print(f"\nTotal images found: {len(all_images)}")

# Display the first image if any were found across all datasets
if all_images:
    first_image_path = all_images[0]
    print(f"First image found: {first_image_path}")
    try:
        img = PIL.Image.open(str(first_image_path))
        img.show()  # This will display the image in a new window
        print(f"Image size: {img.size}")
    except Exception as e:
        print(f"Error opening image: {e}")
else:
    print("No images found across any of the datasets. Check dataset structures.")

In [ ]:
import pathlib
from collections import defaultdict

def organize_images_by_class(root_dir, image_extensions=['*.jpg', '*.jpeg', '*.png']):
    """Organizes images by class based on subdirectory names."""
    root_path = pathlib.Path(root_dir)
    return defaultdict(list, {
        item.name: [p for ext in image_extensions for p in item.glob(ext)]
        for item in root_path.iterdir() if item.is_dir()
    })

if extracted_dirs:
    all_organized_images = defaultdict(list)
    list(map(lambda organized_dataset: [all_organized_images[cls].extend(imgs)
                                         for cls, imgs in organized_dataset.items()],
             map(organize_images_by_class, extracted_dirs)))

    print("\nOrganized images by class:") if all_organized_images else print("\nNo images organized by class found.")
    list(map(lambda item: print(f"Class: {item[0]}, Found {len(item[1])} images"),
             all_organized_images.items()))
else:
    print("\nNo dataset directories found.")

In [ ]:
# Define the CNN model
model = Sequential([
    layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),  # Corrected from flatten() to Flatten()
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary
model.summary()

In [ ]:
# Assuming 'all_organized_images' dictionary from the organization step

image_dir = pathlib.Path("./") # Replace with the actual root directory of your datasets

train_ds = tf.keras.utils.image_dataset_from_directory(
    image_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(img_height, img_width),
    validation_split=0.2,
    subset="training",
    seed=123,
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    image_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(img_height, img_width),
    validation_split=0.2,
    subset="validation",
    seed=123,
    batch_size=32
)

# Optional: Configure performance for faster data loading
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Train the model
model.fit(train_ds, validation_data=val_ds, epochs=10)
# Evaluate the model
loss, accuracy = model.evaluate(test_ds)
print(f"Test accuracy: {accuracy:.2f}")

In [ ]:
# This is a simple Gradio app that uses a *trained* model to classify images
# based on the labels it learned during training.
import gradio as gr
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image

# --- Assume you have a trained model and class labels ---
# Replace these placeholders with your actual trained model and labels

# Example of a simple CNN model (replace with your actual model)
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc = nn.Linear(32 * 56 * 56, num_classes) # Adjust input size based on your training image size

    def forward(self, x):
        x = self.maxpool(self.relu(self.conv1(x)))
        x = self.maxpool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56) # Flatten
        x = self.fc(x)
        return x

# Assume you have the number of classes from your training data
num_classes = 10  # Replace with the actual number of classes

# Load your trained model
model = SimpleCNN(num_classes)
# Replace 'path/to/your/trained_model.pth' with the actual path to your saved model
try:
    model.load_state_dict(torch.load('path/to/your/trained_model.pth'))
    model.eval()  # Set the model to evaluation mode
    print("Trained model loaded successfully.")
except FileNotFoundError:
    print("Error: Trained model file not found. Please provide the correct path.")
    exit()

# Assume you have the class labels corresponding to the model's output
# Replace this with your actual list of class labels
class_labels = ['class_0', 'class_1', 'class_2', 'class_3', 'class_4',
                'class_5', 'class_6', 'class_7', 'class_8', 'class_9']

# Define the transformations used during training
# Make sure these are the *same* transformations you applied to your training data
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Or the size you trained on
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Or your training normalization
])

def classify_image(img):
    """Classifies the input image using the loaded trained model."""
    try:
        img = Image.fromarray(img).convert('RGB') # Ensure image is RGB
        img_t = transform(img).unsqueeze(0)

        with torch.no_grad():
            out = model(img_t)
            probabilities = F.softmax(out, dim=1)
            _, predicted_idx = torch.max(probabilities, 1)
            predicted_label = class_labels[predicted_idx.item()]
            confidence = probabilities[0, predicted_idx.item()].item()

        return f"Predicted Class: {predicted_label} (Confidence: {confidence:.4f})"

    except Exception as e:
        return f"Error during classification: {e}"

demo = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="numpy"),
    outputs="text",
    title="Image Classification with Trained Model",
    description="Upload an image to classify it based on the model it was trained on."
)

demo.launch()
)

In [ ]:
## Create a Gradio interface for the image classification model
demo= gr.Interface(
    fn=lambda img: img. inputs= "image"
    outputs="text",
    title="Image Classification Demo", )
launch(show_error=True)
app.launch(show_error=True)

In [ ]:
## Define a custom theme for the Gradio app
gr.themes.builder()
class Seafoam (Base):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.background_color = "#008080"
        self.text_color = "#000000"
        self.button_color = "#800020"
        self.button_text_color = "#000000"
        self.input_background_color = "#FFFFFF"
        self.input_text_color = "#000000"
        self.output_background_color = "#FFFFFF"
        self.output_text_color = "#000000"
#   Create a Gradio Blocks app with the custom theme
    with gr.Blocks(theme=Seafoam(primary_hue="008080", secondary_hue="800020")) as demo:
        gr.Markdown("# Image Classification with ResNet50")
        gr.Markdown("Upload an image to classify it.")
        gr.Markdown ("ask a medical question")
        image_input = gr.Image(type="pil", label="Input Image")
        image_classify_button = gr.Button("Classify Image")
        output_text = gr.Textbox(label="Classification Result")
        textbox = gr.Textbox(label="Ask a medical question")
        text_classify_button = gr.Button("Ask")
        text_classify_button.click(predict_fn, inputs=image_input, outputs=output_text)
        text_classify_button.click(predict_fn, inputs=textbox, outputs=output_text)
# Set spacing and radius sizes for the Blocks app
with gr.Blocks(theme=gr.themes.Default(spacing_size="md", radius_size="md")) as demo:
# Set fonts for the Blocks app
with gr.Blocks(theme=gr.themes.Default(font=[gr.themes.GoogleFont("Helvetica"), "Arial", "sans-serif"])) as demo: